In [ ]:
'''
def prob(df):
    #  probably calcluation for binary classification
    count = np.count_nonzero(df)   
    if count == 0:
        return float(0)
    else:
        total = len(df)
        prob = np.divide(count,total)
        return float(prob)

prob_udf = udf(prob,FloatType())

OLD VERSION OF new_split, please refer to new_split_2


def split_node(df,feature_array):

    #init
    y_label = df.columns[-1]
    node_parent = {}
    best_gain = -1
    best_feature= 0 
    best_split = 0
    
    #get first tree
    feature_df = feature_split(df,feature_array)
    feature_list = feature_df.rdd.collect()
    
    #init
    for feature in feature_list:
        
        #assign val
        feature_idx = feature[0]
        split_val = feature[1]
        gain = feature[2]
        
        if gain is None:
            gain = 0
        
        #check best
        if gain > best_gain:
            best_gain = gain
            best_feature = feature_idx
            best_split = split_val
        
    #generate split
    left_df = df.filter(col(joined_df.columns[best_feature]) <= best_split)
    right_df = df.filter(col(joined_df.columns[best_feature]) > best_split)
        
    #assign dictinary value of; key: feature, split_value, gain, and child
    node_parent['feature'] = best_feature
    node_parent['split_value'] = best_split
    node_parent['gain'] = best_gain
    
    
    return node_parent, left_df, right_df

'''


In [ ]:
'''
def build_decision_tree(df, feature_array, max_depth, current_depth=0):
    
    """ Recursively build the decision tree. """
    #set y label
    label_col = df.columns[-1]
    distinct_count = df.select(label_col).distinct().count()
    
    if df is None or df.rdd.isEmpty():
        return df
    elif current_depth == max_depth:
        # Return the most common label
        most_common_label = df.groupBy(label_col).count().orderBy("count", ascending=False).first()[label_col]
        return df
        #return most_common_label 
    elif distinct_count == 1:
        return {"label": df.select(label_col).first()[0]}
    
    #split node
    node_parent, left_df, right_df = split_node_2(df,feature_array)
    
    
    # Cache potentially reused DataFrames
    if left_df is not None and not left_df.rdd.isEmpty():
        left_df.cache()
    if right_df is not None and not right_df.rdd.isEmpty():
        right_df.cache()

    
    #split if not empty    
    if left_df is None or left_df == []:
        right_subtree = build_decision_tree(right_df, feature_array, max_depth, current_depth + 1)
    elif right_df is None or right_df ==[]:
        left_subtree = build_decision_tree(left_df, feature_array, max_depth, current_depth + 1)
    elif (left_df is None) and (right_df is None):
        return {"feature": node_parent['feature'] , "threshold": node_parent['split_value'], "left": None, "right": None}
    else:
        left_subtree = build_decision_tree(left_df, feature_array, max_depth, current_depth + 1)
        right_subtree = build_decision_tree(right_df, feature_array, max_depth, current_depth + 1)

    # Clear cache after use
    if left_df is not None:
        left_df.unpersist()
    if right_df is not None:
        right_df.unpersist()
        
    
    # return node structure
    return {
        "feature": node_parent['feature'] , 
        "threshold": node_parent['split_value'], 
        "left": left_subtree, 
        "right": right_subtree
    }

#TESTING:
tree = build_decision_tree(joined_df, [0,2,3], max_depth=5)
tree

'''



In [ ]:
'''
def feature_split_2(dataset, feature_array):
    """
    Input: 
    - dataset: a pyspark dataframe partition to be called by foreachPartition,
    - feature_array: a broadcasted feature array for the tree that is initialized earlier on
    """
    # Define schema
    schema = StructType([
        StructField("feature", IntegerType(), True),
        StructField("split_value", FloatType(), True),
        StructField("info_gain", FloatType(), True),
    ])
    
    # Broadcast feature array
    broadcast_feature_array = spark.sparkContext.broadcast(feature_array)
    
    # Register the DataFrame as a temporary view
    dataset.createOrReplaceTempView("dataset")
    
    # Create an empty DataFrame with the defined schema
    result_df = spark.createDataFrame([], schema)
    
    for feature_index in broadcast_feature_array.value:
        feature_col_name = dataset.columns[feature_index]
        
        # Use SQL to find split for the feature
        split_result = spark.sql(f"""
            WITH distinct_values AS (
                SELECT DISTINCT {feature_col_name} AS split_value
                FROM dataset
                ORDER BY RAND()
                LIMIT 10
            ),
            left_counts AS (
                SELECT split_value, COUNT(*) AS left_count, SUM(y) AS left_sum
                FROM dataset, distinct_values
                WHERE {feature_col_name} <= split_value
                GROUP BY split_value
            ),
            right_counts AS (
                SELECT split_value, COUNT(*) AS right_count, SUM(y) AS right_sum
                FROM dataset, distinct_values
                WHERE {feature_col_name} > split_value
                GROUP BY split_value
            )
            SELECT
                {feature_index} AS feature,
                d.split_value,
                (left_counts.left_count / (left_counts.left_count + right_counts.right_count)) * (
                    -(left_counts.left_sum / left_counts.left_count) * LOG2(left_counts.left_sum / left_counts.left_count) - 
                    (1 - left_counts.left_sum / left_counts.left_count) * LOG2(1 - left_counts.left_sum / left_counts.left_count)
                ) +
                (right_counts.right_count / (left_counts.left_count + right_counts.right_count)) * (
                    -(right_counts.right_sum / right_counts.right_count) * LOG2(right_counts.right_sum / right_counts.right_count) - 
                    (1 - right_counts.right_sum / right_counts.right_count) * LOG2(1 - right_counts.right_sum / right_counts.right_count)
                ) AS info_gain
            FROM distinct_values d
            JOIN left_counts ON d.split_value = left_counts.split_value
            JOIN right_counts ON d.split_value = right_counts.split_value
            ORDER BY info_gain DESC
            LIMIT 1
        """)
        
        # Append the split result to the result DataFrame
        result_df = result_df.union(split_result.withColumn("feature", F.lit(feature_index)))
    
    return result_df
'''

In [ ]:
'''
# Build tree from splitting

# each tree
# (i) for each feature: find_split
# (ii) Mapbypartition(find_split)

def feature_split(dataset, feature_array):
    

    Input: 
    partition: a pyspark dataframe partition to be called by foreachPartition,
    feature_array: a broadcasted feature array for the tree that is intiialized earlier on
    #define schema
    schema = StructType([
        StructField("feature", IntegerType(), True),
        StructField("split_value", FloatType(), True),
        StructField("info_gain", FloatType(), True),
    ])
    feature_df = spark.createDataFrame([], schema)
    
    # for each feature array, get a split and append the dataframe 
    for feature_index in feature_array:
        
        # find split
        feature_split = new_split_2(dataset, feature_index)
        
        #add feature  
        feature_df = feature_df.union(feature_split)
        
    return feature_df
'''